# Bias Mitigation Experiments - Bank Marketing

## Table of Contents

In [21]:
data_name = 'bank+marketing'

In [22]:
# to ignore warnings
import warnings
# warnings.filterwarnings('ignore')

In [23]:
import numpy as np
import pandas as pd
import tensorflow.compat.v1 as tf
# tf.disable_eager_execution() # comment out when running TabTransformer models

## [1.](#Table-of-Contents) Prepare Bank+marketing data

In [24]:
# read in data as pandas data frame
df = pd.read_csv("../data/bank+marketing/bank/bank-full.csv", sep=';')
# check columns
pd.set_option('display.max_columns', None) # expand all columns
print(df.shape)
df.head()

(45211, 17)


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [25]:
# data pre-processing
import re
label = 'y'
protected_attribute = 'age'
# # 1. select features of interest
# keep_var = ['sex','age','race','juv_fel_count','juv_misd_count','juv_other_count','priors_count',
#             'c_charge_degree', 'two_year_recid']
# df = df[keep_var]

# 2. encode label and potential protected features 
df.age = df.age.apply(lambda x: 0 if x<=25 else 1)
df = df.reset_index(drop=True)

# 3. covert to binary variables if possible
binary_var = ['default', 'housing', 'loan', 'y']
binary_map = {'yes':1, 'no':0}
df[binary_var] = df[binary_var].replace(binary_map)

print(df.shape)
df.head()

(45211, 17)


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,1,management,married,tertiary,0,2143,1,0,unknown,5,may,261,1,-1,0,unknown,0
1,1,technician,single,secondary,0,29,1,0,unknown,5,may,151,1,-1,0,unknown,0
2,1,entrepreneur,married,secondary,0,2,1,1,unknown,5,may,76,1,-1,0,unknown,0
3,1,blue-collar,married,unknown,0,1506,1,0,unknown,5,may,92,1,-1,0,unknown,0
4,1,unknown,single,unknown,0,1,0,0,unknown,5,may,198,1,-1,0,unknown,0


In [26]:
# # flip labels to ensure pos_label = 1, not becoming recidivist
# df[label] = (~df[label].astype(bool)).astype(int)

# select descriptive features and target variable
X = df.drop(columns=label, axis=1) # select all features but target feature
y = df[[protected_attribute, label]] # include protected feature in order to AIF360

In [27]:
# set protected attribute as index
X = X.set_index([protected_attribute], append = True, drop = False)
y = y.set_index([protected_attribute], append = True)

# make y data frames to 1d array to pass modeling, but keep index (protected attribute)
y = pd.Series(y[label], index=y.index)

In [28]:
X.to_pickle('../data/{}/{}_X'.format(data_name, data_name))
y.to_pickle('../data/{}/{}_y'.format(data_name, data_name))

## [2.](#Table-of-Contents) Proposed GridSearch Approach 

The GridSearch Approach includes hyperparameter, threshold, and Bias Mitigation

In [29]:
from FairGridSearch import *
%load_ext autoreload
%autoreload 2
# allow automatic reloading of changes in FairGridSearch file

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
# None, better
# 'RW', worse
# 'LFR_pre', stuck
# 'LFR_in', stuck
# 'AD', stuck
# 'EGR',better
# 'ROC', better
# 'CEO', better

## 3. Case Study: Bank + Marketing

In [31]:
label = 'two_year_recid'
protected_attribute = 'age'

pos_label = 0
priv_group = 1

cv=10
n_jobs=multiprocessing.cpu_count()-1

In [32]:
X = pd.read_pickle('../data/{}/{}_X'.format(data_name, data_name))
y = pd.read_pickle('../data/{}/{}_y'.format(data_name, data_name))

In [33]:
# define desired metric for the use case
ACC_METRIC = 'avg_norm_mcc_score'
FAIR_METRIC = 'abs_avg_ppvd_score'

### Logistic Regression

In [13]:
import timeit
start = timeit.default_timer()

param_grid = {'hyperp_grid': {'C':[1, 10],'solver':['liblinear', 'saga'],'penalty':['l2']}, 
              'threshold': np.linspace(0.3, 0.7, 5),
              'Bias_Mitigation':[None,'RW','LFR_pre','LFR_in','AD','EGR','ROC','CEO','RW+ROC','RW+CEO']}

clf_lr = fair_GridsearchCV(base='LR', param_grid=param_grid, 
                           prot_attr=protected_attribute, pos_label=pos_label, priv_group=priv_group,
                           cv=cv, n_jobs=n_jobs)
if __name__ == '__main__':
    clf_lr.fit(X=X, y=y)
    
results_lr = clf_lr.output_table
print(clf_lr._best_param)

stop = timeit.default_timer()
runtime = stop - start
if runtime < 60:
    print('Time: ', runtime, 'sec')
else: print('Time: ', runtime/60, 'min')
results_lr.to_pickle('./{}_results/{}_results_LR'.format(data_name,data_name))
style_table(results_lr)

  0%|          | 0/4 [00:00<?, ?it/s]

{'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}
------------------------------------------------------------------------------------------
start multiprocessing
------------------------------------------------------------------------------------------


 25%|██▌       | 1/4 [12:15<36:45, 735.25s/it]

{'C': 1, 'penalty': 'l2', 'solver': 'saga'}
------------------------------------------------------------------------------------------
start multiprocessing
------------------------------------------------------------------------------------------


 50%|█████     | 2/4 [25:16<25:24, 762.14s/it]

...

 75%|███████▌  | 3/4 [36:33<12:03, 723.20s/it]

{'C': 10, 'penalty': 'l2', 'solver': 'saga'}
------------------------------------------------------------------------------------------
start multiprocessing
------------------------------------------------------------------------------------------


100%|██████████| 4/4 [49:48<00:00, 747.08s/it]


base_estimator                                                   LR
param              {'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}
Bias_Mitigation                                                  AD
threshold                                                       0.4
Name: 21, dtype: object
Time:  49.81662042 min


OSError: Cannot save file into a non-existent directory: 'bank+marketing_results'

In [18]:
results_lr.to_pickle('./{}_results/{}_results_LR'.format(data_name,data_name))
style_table(results_lr)

,base_estimator,param,Bias_Mitigation,threshold,avg_acc_score,avg_bacc_score,avg_f1_score,avg_auc_score,avg_mcc_score,avg_norm_mcc_score,avg_spd_score,avg_aod_score,avg_eod_score,avg_ford_score,avg_ppvd_score,avg_(1-consistency_score),avg_gei_score,avg_ti_score,cost
0,LR,"{'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}",None,0.300000,0.900400,0.746121,0.561180,0.907377,0.505453,0.752726,-0.175658,-0.120783,-0.105506,0.005467,-0.048798,0.064307,0.050471,0.071634,0.422932
1,LR,"{'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}",None,0.400000,0.903165,0.701266,0.513876,0.907377,0.470845,0.735422,-0.141027,-0.103134,-0.077655,0.013298,-0.064187,0.050598,0.051314,0.079942,0.405605
2,LR,"{'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}",None,0.500000,0.901772,0.660866,0.451932,0.907377,0.427845,0.713922,-0.107666,-0.088308,-0.048639,-0.014930,-0.073247,0.040680,0.053286,0.088170,0.393743
3,LR,"{'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}",None,0.600000,0.899869,0.628542,0.390420,0.907377,0.388965,0.694482,-0.084462,-0.075601,-0.031209,-0.034964,-0.081359,0.031966,0.055040,0.094905,0.389979
4,LR,"{'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}",None,0.700000,0.896463,0.597828,0.319201,0.907377,0.340791,0.670396,-0.057683,-0.045030,-0.022021,-0.024892,-0.098384,0.025073,0.057433,0.101741,0.387287
5,LR,"{'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}",RW,0.300000,0.900599,0.744675,0.560066,0.905718,0.504578,0.752289,-0.052820,0.035438,-0.013989,-0.107596,-0.095762,0.063383,0.050469,0.071884,0.300531
6,LR,"{'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}",RW,0.400000,0.903475,0.700540,0.513500,0.905718,0.471232,0.735616,-0.042305,0.030378,-0.010347,-0.092242,-0.103052,0.049806,0.051203,0.080009,0.306689
7,LR,"{'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}",RW,0.500000,0.901263,0.658610,0.447396,0.905718,0.423637,0.711818,-0.023122,0.046762,-0.007384,-0.058266,-0.117663,0.040495,0.053598,0.088740,0.311303
8,LR,"{'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}",RW,0.600000,0.899117,0.625739,0.383929,0.905718,0.382625,0.691313,-0.013424,0.042512,-0.002702,-0.076833,-0.121096,0.031762,0.055493,0.095653,0.322112
9,LR,"{'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}",RW,0.700000,0.896198,0.596037,0.314748,0.905718,0.337525,0.668763,-0.006996,0.040349,-0.002381,-0.048615,-0.125397,0.024764,0.057607,0.102159,0.338233


### Random Forest

In [16]:
import timeit
start = timeit.default_timer()

param_grid = {'hyperp_grid': {'n_estimators':[10, 50],'criterion':['gini', 'entropy'], 'max_depth':[16]}, 
              'threshold': np.linspace(0.3, 0.7, 5),
              'Bias_Mitigation':[None,'RW','LFR_pre','EGR','ROC','CEO','RW+ROC','RW+CEO']}

clf_rf = fair_GridsearchCV(base='RF', param_grid=param_grid,
                           prot_attr=protected_attribute, pos_label=pos_label, priv_group=priv_group,
                           cv=cv, n_jobs=n_jobs)
if __name__ == '__main__':
    clf_rf.fit(X=X, y=y)
    
results_rf = clf_rf.output_table
print(clf_rf._best_param)

stop = timeit.default_timer()
runtime = stop - start
if runtime < 60:
    print('Time: ', runtime, 'sec')
else: print('Time: ', runtime/60, 'min')
results_rf.to_pickle('./{}_results/{}_results_RF'.format(data_name, data_name))
style_table(results_rf)

  0%|          | 0/4 [00:00<?, ?it/s]

{'criterion': 'gini', 'max_depth': 16, 'n_estimators': 10}
------------------------------------------------------------------------------------------
start multiprocessing
------------------------------------------------------------------------------------------


 25%|██▌       | 1/4 [06:39<19:59, 399.98s/it]

{'criterion': 'gini', 'max_depth': 16, 'n_estimators': 50}
------------------------------------------------------------------------------------------
start multiprocessing
------------------------------------------------------------------------------------------


 50%|█████     | 2/4 [16:41<17:16, 518.45s/it]

...

 75%|███████▌  | 3/4 [23:33<07:50, 470.11s/it]

{'criterion': 'entropy', 'max_depth': 16, 'n_estimators': 50}
------------------------------------------------------------------------------------------
start multiprocessing
------------------------------------------------------------------------------------------


100%|██████████| 4/4 [34:11<00:00, 512.76s/it]


base_estimator                                                    RF
param              {'criterion': 'gini', 'max_depth': 16, 'n_esti...
Bias_Mitigation                                                  ROC
threshold                                                        0.5
Name: 22, dtype: object
Time:  34.19267995 min


,base_estimator,param,Bias_Mitigation,threshold,avg_acc_score,avg_bacc_score,avg_f1_score,avg_auc_score,avg_mcc_score,avg_norm_mcc_score,avg_spd_score,avg_aod_score,avg_eod_score,avg_ford_score,avg_ppvd_score,avg_(1-consistency_score),avg_gei_score,avg_ti_score,cost
0,RF,"{'criterion': 'gini', 'max_depth': 16, 'n_estimators': 10}",None,0.300000,0.892924,0.790027,0.588974,0.917688,0.531987,0.765994,-0.170727,-0.084743,-0.110727,-0.015050,-0.055352,0.095339,0.050442,0.064017,0.404733
1,RF,"{'criterion': 'gini', 'max_depth': 16, 'n_estimators': 10}",None,0.400000,0.900975,0.731189,0.546192,0.917688,0.492756,0.746378,-0.110754,-0.049170,-0.057987,-0.034753,-0.076783,0.072699,0.050964,0.074504,0.364376
2,RF,"{'criterion': 'gini', 'max_depth': 16, 'n_estimators': 10}",None,0.500000,0.902657,0.668665,0.466067,0.917688,0.439034,0.719517,-0.084368,-0.047992,-0.039620,-0.015855,-0.088210,0.051231,0.052621,0.086421,0.364851
3,RF,"{'criterion': 'gini', 'max_depth': 16, 'n_estimators': 10}",None,0.600000,0.899339,0.609461,0.349130,0.917688,0.369287,0.684644,-0.059055,-0.047700,-0.020234,-0.041366,-0.095344,0.031966,0.055660,0.098684,0.374411
4,RF,"{'criterion': 'gini', 'max_depth': 16, 'n_estimators': 10}",None,0.700000,0.893942,0.562447,0.222229,0.917688,0.290447,0.645223,-0.029668,-0.026039,-0.004968,-0.076032,-0.107764,0.018283,0.059272,0.109183,0.384445
5,RF,"{'criterion': 'gini', 'max_depth': 16, 'n_estimators': 10}",RW,0.300000,0.893323,0.789595,0.589388,0.915152,0.532373,0.766186,-0.106723,-0.009350,-0.052669,-0.074763,-0.073514,0.094928,0.050349,0.064040,0.340537
6,RF,"{'criterion': 'gini', 'max_depth': 16, 'n_estimators': 10}",RW,0.400000,0.902103,0.731498,0.548556,0.915152,0.496221,0.748110,-0.077503,-0.007867,-0.029309,-0.093937,-0.085379,0.072500,0.050494,0.074194,0.329393
7,RF,"{'criterion': 'gini', 'max_depth': 16, 'n_estimators': 10}",RW,0.500000,0.901971,0.666472,0.461067,0.915152,0.433731,0.716865,-0.051892,-0.009578,-0.013535,-0.114418,-0.096496,0.051045,0.053029,0.087033,0.335027
8,RF,"{'criterion': 'gini', 'max_depth': 16, 'n_estimators': 10}",RW,0.600000,0.897812,0.605563,0.338655,0.915152,0.356514,0.678257,-0.037974,-0.016642,-0.009703,-0.094611,-0.105179,0.031245,0.056563,0.099873,0.359717
9,RF,"{'criterion': 'gini', 'max_depth': 16, 'n_estimators': 10}",RW,0.700000,0.893721,0.564291,0.227920,0.915152,0.290146,0.645073,-0.021409,-0.009726,-0.003282,-0.125102,-0.113192,0.018279,0.059389,0.108906,0.376336


### Gradient Boosting

In [19]:
import timeit
start = timeit.default_timer()

param_grid = {'hyperp_grid': {'n_estimators':[10, 50],'criterion':['friedman_mse'],'max_depth':[8, 32]}, 
              'threshold': np.linspace(0.3, 0.7, 5),
              'Bias_Mitigation':[None,'RW','LFR_pre','EGR','ROC','CEO','RW+ROC','RW+CEO']}

clf_gb = fair_GridsearchCV(base='GB', param_grid=param_grid,
                           prot_attr=protected_attribute, pos_label=pos_label, priv_group=priv_group,
                           cv=cv, n_jobs=n_jobs)
if __name__ == '__main__':
    clf_gb.fit(X=X, y=y)
    
results_gb = clf_gb.output_table
print(clf_gb._best_param)

stop = timeit.default_timer()
runtime = stop - start
if runtime < 60:
    print('Time: ', runtime, 'sec')
else: print('Time: ', runtime/60, 'min')
results_gb.to_pickle('./{}_results/{}_results_GB'.format(data_name, data_name))
style_table(results_gb)

  0%|          | 0/4 [00:00<?, ?it/s]

{'criterion': 'friedman_mse', 'max_depth': 8, 'n_estimators': 10}
------------------------------------------------------------------------------------------
start multiprocessing
------------------------------------------------------------------------------------------


 25%|██▌       | 1/4 [09:32<28:37, 572.34s/it]

{'criterion': 'friedman_mse', 'max_depth': 8, 'n_estimators': 50}
------------------------------------------------------------------------------------------
start multiprocessing
------------------------------------------------------------------------------------------


 50%|█████     | 2/4 [30:21<32:21, 970.57s/it]

...

 75%|███████▌  | 3/4 [1:06:13<25:09, 1509.85s/it]

{'criterion': 'friedman_mse', 'max_depth': 32, 'n_estimators': 50}
------------------------------------------------------------------------------------------
start multiprocessing
------------------------------------------------------------------------------------------


100%|██████████| 4/4 [7:04:31<00:00, 6367.87s/it]


base_estimator                                                    GB
param              {'criterion': 'friedman_mse', 'max_depth': 8, ...
Bias_Mitigation                                               RW+ROC
threshold                                                        0.5
Name: 72, dtype: object
Time:  424.5317487816666 min


,base_estimator,param,Bias_Mitigation,threshold,avg_acc_score,avg_bacc_score,avg_f1_score,avg_auc_score,avg_mcc_score,avg_norm_mcc_score,avg_spd_score,avg_aod_score,avg_eod_score,avg_ford_score,avg_ppvd_score,avg_(1-consistency_score),avg_gei_score,avg_ti_score,cost
0,GB,"{'criterion': 'friedman_mse', 'max_depth': 8, 'n_estimators': 10}",None,0.300000,0.903541,0.739860,0.560715,0.914723,0.508352,0.754176,-0.086422,-0.005057,-0.040999,-0.038500,-0.086419,0.073318,0.049526,0.072206,0.332246
1,GB,"{'criterion': 'friedman_mse', 'max_depth': 8, 'n_estimators': 10}",None,0.400000,0.902126,0.664675,0.458350,0.914723,0.432698,0.716349,-0.059880,-0.015275,-0.019878,-0.056860,-0.094424,0.048192,0.053002,0.087340,0.343531
2,GB,"{'criterion': 'friedman_mse', 'max_depth': 8, 'n_estimators': 10}",None,0.500000,0.896574,0.590510,0.301098,0.914723,0.334607,0.667304,-0.033014,-0.011975,-0.009991,-0.048504,-0.109387,0.025361,0.057472,0.103094,0.365710
3,GB,"{'criterion': 'friedman_mse', 'max_depth': 8, 'n_estimators': 10}",None,0.600000,0.889076,0.534679,0.131688,0.914723,0.215035,0.607517,-0.015231,-0.011364,-0.003350,-0.072003,-0.117568,0.012024,0.062367,0.115935,0.407713
4,GB,"{'criterion': 'friedman_mse', 'max_depth': 8, 'n_estimators': 10}",None,0.700000,0.883214,0.501179,0.004891,0.914723,0.032271,0.516135,0.000387,0.001354,0.000103,-0.250000,-0.126309,0.000464,0.066114,0.124126,0.484252
5,GB,"{'criterion': 'friedman_mse', 'max_depth': 8, 'n_estimators': 10}",RW,0.300000,0.903342,0.738272,0.558499,0.914735,0.506217,0.753108,-0.042873,0.053515,-0.011488,-0.086728,-0.103902,0.072058,0.049682,0.072569,0.289764
6,GB,"{'criterion': 'friedman_mse', 'max_depth': 8, 'n_estimators': 10}",RW,0.400000,0.902457,0.667405,0.463528,0.914735,0.436835,0.718418,-0.020437,0.051570,-0.005384,-0.073062,-0.117847,0.048382,0.052758,0.086721,0.302020
7,GB,"{'criterion': 'friedman_mse', 'max_depth': 8, 'n_estimators': 10}",RW,0.500000,0.897260,0.592458,0.306709,0.914735,0.341148,0.670574,-0.016153,0.021089,-0.004822,-0.069346,-0.119644,0.025578,0.057059,0.102520,0.345579
8,GB,"{'criterion': 'friedman_mse', 'max_depth': 8, 'n_estimators': 10}",RW,0.600000,0.887616,0.530244,0.116911,0.914735,0.191222,0.595611,-0.000855,0.016060,-0.000617,-0.140550,-0.126828,0.011639,0.063285,0.117237,0.405244
9,GB,"{'criterion': 'friedman_mse', 'max_depth': 8, 'n_estimators': 10}",RW,0.700000,0.883391,0.501935,0.007877,0.914735,0.047249,0.523625,0.000570,0.002174,0.000103,-0.136667,-0.126472,0.000739,0.066001,0.123926,0.476946


### Support Vector Machine

In [ ]:
import timeit
start = timeit.default_timer()

param_grid = {'hyperp_grid': {'kernel':['rbf','linear','poly','sigmoid'],'gamma':['scale']},
              'threshold': np.linspace(0.3, 0.7, 5),
              'Bias_Mitigation':[None,'RW','LFR_pre','EGR','ROC','CEO','RW+ROC','RW+CEO']}

clf_svm = fair_GridsearchCV(base='SVM', param_grid=param_grid,
                            prot_attr=protected_attribute, pos_label=pos_label, priv_group=priv_group,
                            cv=cv, n_jobs=n_jobs)
if __name__ == '__main__':
    clf_svm.fit(X=X, y=y)
    
results_svm = clf_svm.output_table
print(clf_svm._best_param)

stop = timeit.default_timer()
runtime = stop - start
if runtime < 60:
    print('Time: ', runtime, 'sec')
else: print('Time: ', runtime/60, 'min')
results_svm.to_pickle('./{}_results/{}_results_SVM'.format(data_name, data_name))
style_table(results_svm)

  0%|          | 0/4 [00:00<?, ?it/s]

{'gamma': 'scale', 'kernel': 'rbf'}
running Bias Mitigation None
running Bias Mitigation None
running Bias Mitigation None
------------------------------------------------------------------------------------------
start multiprocessing
------------------------------------------------------------------------------------------
running Bias Mitigation RW
running Bias Mitigation RW
running Bias Mitigation RW
running Bias Mitigation LFR_pre
running Bias Mitigation LFR_pre
running Bias Mitigation LFR_pre


/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: False Omission Rate is ill-defined and being set to 0.0 due to no predicted negative samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: False Omission Rate is ill-defined and being set to 0.0 due to no predicted negative samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: False Omission Rate is ill-defined and being set to 0.0 due to no predicted negative samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

running Bias Mitigation EGR


/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/fairlearn/reductions/_moments/utility_parity.py:251: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  lambda_event = (lambda_vec["+"] - self.ratio * lambda_vec["-"]).sum(level=_EVENT) / \
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: False Omission Rate is ill-defined and being set to 0.0 due to no predicted negative samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: False Omission Rate is ill-defined and being set to 0.0 due to no predicted negative samples. Use `zero_divi

...

/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/fairlearn/reductions/_moments/utility_parity.py:251: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  lambda_event = (lambda_vec["+"] - self.ratio * lambda_vec["-"]).sum(level=_EVENT) / \
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/fairlearn/reductions/_moments/utility_parity.py:251: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  lambda_event = (lambda_vec["+"] - self.ratio * lambda_vec["-"]).sum(level=_EVENT) / \
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: False Omission Rate is ill-defi

running Bias Mitigation LFR_pre


/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/fairlearn/reductions/_moments/utility_parity.py:251: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  lambda_event = (lambda_vec["+"] - self.ratio * lambda_vec["-"]).sum(level=_EVENT) / \
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/fairlearn/reductions/_moments/utility_parity.py:251: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  lambda_event = (lambda_vec["+"] - self.ratio * lambda_vec["-"]).sum(level=_EVENT) / \


running Bias Mitigation EGR


/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/fairlearn/reductions/_moments/utility_parity.py:251: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  lambda_event = (lambda_vec["+"] - self.ratio * lambda_vec["-"]).sum(level=_EVENT) / \
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/fairlearn/reductions/_moments/utility_parity.py:251: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  lambda_event = (lambda_vec["+"] - self.ratio * lambda_vec["-"]).sum(level=_EVENT) / \
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/fairlearn/reductions/_moments/utility_parity.py:251: FutureWarning: Using the level keyword in D

### Gaussian Naive Bayes

In [15]:
# model for Reweighing
import timeit
start = timeit.default_timer()

param_grid = {'hyperp_grid': {'var_smoothing': np.logspace(0,-9, num=4)}, 
              'threshold': np.linspace(0.3, 0.7, 5),
              'Bias_Mitigation':[None,'RW','LFR_pre','EGR','ROC','CEO','RW+ROC','RW+CEO']}

clf_nb = fair_GridsearchCV(base='NB', param_grid=param_grid, 
                           prot_attr=protected_attribute, pos_label=pos_label, priv_group=priv_group,
                           cv=cv, n_jobs=n_jobs)
if __name__ == '__main__':
    clf_nb.fit(X=X, y=y)
    
results_nb = clf_nb.output_table
# print(clf._best_param)

stop = timeit.default_timer()
runtime = stop - start
if runtime < 60:
    print('Time: ', runtime, 'sec')
else: print('Time: ', runtime/60, 'min')
results_nb.to_pickle('./{}_results/{}_results_NB'.format(data_name, data_name))
style_table(results_nb)

  0%|          | 0/4 [00:00<?, ?it/s]

{'var_smoothing': 1.0}
------------------------------------------------------------------------------------------
start multiprocessing
------------------------------------------------------------------------------------------


 25%|██▌       | 1/4 [06:00<18:02, 360.73s/it]

{'var_smoothing': 0.001}
------------------------------------------------------------------------------------------
start multiprocessing
------------------------------------------------------------------------------------------


 50%|█████     | 2/4 [12:42<12:49, 384.61s/it]

...

 75%|███████▌  | 3/4 [19:02<06:22, 382.88s/it]

{'var_smoothing': 1e-09}
------------------------------------------------------------------------------------------
start multiprocessing
------------------------------------------------------------------------------------------


100%|██████████| 4/4 [25:24<00:00, 381.05s/it]


Time:  25.41080901 min


,base_estimator,param,Bias_Mitigation,threshold,avg_acc_score,avg_bacc_score,avg_f1_score,avg_auc_score,avg_mcc_score,avg_norm_mcc_score,avg_spd_score,avg_aod_score,avg_eod_score,avg_ford_score,avg_ppvd_score,avg_(1-consistency_score),avg_gei_score,avg_ti_score,cost
0,NB,{'var_smoothing': 1.0},None,0.300000,0.876667,0.664201,0.423227,0.822964,0.356917,0.678459,-0.369753,-0.346671,-0.317183,0.093930,-0.037112,0.011710,0.064008,0.093335,0.691295
1,NB,{'var_smoothing': 1.0},None,0.400000,0.877618,0.660967,0.419563,0.822964,0.354840,0.677420,-0.338812,-0.314478,-0.287660,0.088926,-0.046876,0.010245,0.063832,0.093816,0.661391
2,NB,{'var_smoothing': 1.0},None,0.500000,0.878370,0.657538,0.415264,0.822964,0.352070,0.676035,-0.324270,-0.302674,-0.272103,0.084687,-0.048788,0.008816,0.063735,0.094374,0.648234
3,NB,{'var_smoothing': 1.0},None,0.600000,0.879211,0.655308,0.412840,0.822964,0.351147,0.675573,-0.319664,-0.298035,-0.268725,0.089924,-0.051646,0.007892,0.063525,0.094658,0.644091
4,NB,{'var_smoothing': 1.0},None,0.700000,0.879874,0.653305,0.410450,0.822964,0.350069,0.675035,-0.308193,-0.282467,-0.261989,0.098517,-0.059651,0.007184,0.063369,0.094932,0.633159
5,NB,{'var_smoothing': 1.0},RW,0.300000,0.881312,0.665109,0.430115,0.823488,0.368572,0.684286,-0.181799,-0.152483,-0.129396,-0.001090,-0.069097,0.014421,0.062161,0.092187,0.497513
6,NB,{'var_smoothing': 1.0},RW,0.400000,0.882108,0.658507,0.421135,0.823488,0.362184,0.681092,-0.155972,-0.122871,-0.110714,-0.005812,-0.079652,0.011864,0.062192,0.093377,0.474880
7,NB,{'var_smoothing': 1.0},RW,0.500000,0.882949,0.654718,0.416284,0.823488,0.359529,0.679765,-0.137686,-0.102756,-0.094535,-0.013936,-0.083388,0.010254,0.062039,0.093969,0.457922
8,NB,{'var_smoothing': 1.0},RW,0.600000,0.883811,0.651516,0.412205,0.823488,0.357668,0.678834,-0.122638,-0.088541,-0.079912,-0.028477,-0.085451,0.009334,0.061838,0.094431,0.443804
9,NB,{'var_smoothing': 1.0},RW,0.700000,0.884630,0.648700,0.408569,0.823488,0.356233,0.678116,-0.110139,-0.076292,-0.068872,-0.035750,-0.088147,0.007781,0.061625,0.094820,0.432023


### TabTransformer

In [34]:
import timeit
start = timeit.default_timer()

param_grid = {'hyperp_grid': {'epochs':[20, 30],'learing_rate':[1e-04, 1e-05]},
              'threshold': np.linspace(0.3, 0.7, 5),
              'Bias_Mitigation':[None,'RW','ROC','CEO','RW+ROC','RW+CEO']}

clf_tab = fair_GridsearchCV(base='TabTrans',param_grid=param_grid, 
                            prot_attr=protected_attribute, pos_label=pos_label, priv_group=priv_group,
                            cv=cv, n_jobs=n_jobs)

clf_tab.fit(X=X, y=y)
results_tab = clf_tab.output_table
# print(clf._best_param)

stop = timeit.default_timer()
runtime = stop - start
if runtime < 60:
    print('Time: ', runtime, 'sec')
else: print('Time: ', runtime/60, 'min')
results_tab.to_pickle('./{}_results/{}_results_TabTrans'.format(data_name, data_name))

style_table(results_tab)

  0%|          | 0/4 [00:00<?, ?it/s]

{'epochs': 20, 'learing_rate': 0.0001}
------------------------------------------------------------------------------------------
start multiprocessing
------------------------------------------------------------------------------------------


 25%|██▌       | 1/4 [3:01:54<9:05:44, 10914.73s/it]

{'epochs': 20, 'learing_rate': 1e-05}
------------------------------------------------------------------------------------------
start multiprocessing
------------------------------------------------------------------------------------------


 50%|█████     | 2/4 [6:01:31<6:01:07, 10833.77s/it]

...

 75%|███████▌  | 3/4 [10:03:58<3:28:49, 12529.17s/it]

{'epochs': 30, 'learing_rate': 1e-05}
------------------------------------------------------------------------------------------
start multiprocessing
------------------------------------------------------------------------------------------


100%|██████████| 4/4 [14:06:31<00:00, 12697.95s/it]  


Time:  846.5372749683333 min


,base_estimator,param,Bias_Mitigation,threshold,avg_acc_score,avg_bacc_score,avg_f1_score,avg_auc_score,avg_mcc_score,avg_norm_mcc_score,avg_spd_score,avg_aod_score,avg_eod_score,avg_ford_score,avg_ppvd_score,avg_(1-consistency_score),avg_gei_score,avg_ti_score,cost
0,TabTrans,"{'epochs': 20, 'learing_rate': 0.0001}",None,0.300000,0.886023,0.658507,0.424822,0.836264,0.372127,0.686063,-0.134495,-0.114674,-0.080086,-0.051168,-0.073150,0.097561,0.060526,0.092490,0.448432
1,TabTrans,"{'epochs': 20, 'learing_rate': 0.0001}",None,0.400000,0.892880,0.630161,0.384876,0.836264,0.359892,0.679946,-0.100860,-0.089542,-0.052094,-0.037795,-0.083239,0.069802,0.058541,0.096489,0.420914
2,TabTrans,"{'epochs': 20, 'learing_rate': 0.0001}",None,0.500000,0.894340,0.602858,0.329475,0.836264,0.334790,0.667395,-0.066728,-0.053667,-0.031004,-0.023304,-0.097358,0.049320,0.058495,0.101391,0.399333
3,TabTrans,"{'epochs': 20, 'learing_rate': 0.0001}",None,0.600000,0.894583,0.580853,0.274964,0.836264,0.312011,0.656005,-0.048809,-0.043553,-0.016978,-0.031290,-0.101428,0.034912,0.058724,0.105519,0.392804
4,TabTrans,"{'epochs': 20, 'learing_rate': 0.0001}",None,0.700000,0.891730,0.554387,0.197208,0.836264,0.262616,0.631308,-0.023156,-0.015867,-0.005577,-0.072624,-0.113383,0.022862,0.060649,0.111391,0.391848
5,TabTrans,"{'epochs': 20, 'learing_rate': 0.0001}",RW,0.300000,0.881754,0.681760,0.453840,0.855690,0.389690,0.694845,-0.147353,-0.113059,-0.093784,-0.045222,-0.071546,0.114388,0.061072,0.088657,0.452508
6,TabTrans,"{'epochs': 20, 'learing_rate': 0.0001}",RW,0.400000,0.891376,0.641857,0.404826,0.855690,0.368228,0.684114,-0.106893,-0.088804,-0.055651,-0.050221,-0.080528,0.078897,0.058882,0.094573,0.422779
7,TabTrans,"{'epochs': 20, 'learing_rate': 0.0001}",RW,0.500000,0.895402,0.614530,0.356410,0.855690,0.352851,0.676426,-0.084481,-0.073055,-0.041347,-0.010052,-0.089879,0.055416,0.057689,0.098848,0.408056
8,TabTrans,"{'epochs': 20, 'learing_rate': 0.0001}",RW,0.600000,0.894760,0.586611,0.290308,0.855690,0.319054,0.659527,-0.045671,-0.031748,-0.018882,-0.009517,-0.105759,0.038451,0.058550,0.104375,0.386144
9,TabTrans,"{'epochs': 20, 'learing_rate': 0.0001}",RW,0.700000,0.892548,0.560346,0.215971,0.855690,0.275929,0.637964,-0.033111,-0.026878,-0.011187,-0.043067,-0.109896,0.025118,0.060118,0.110014,0.395147
